# Syntax tutorial

This notebook is a terse tutorial walkthrough of the syntax of the `engine` language (which is based on the Handlebars templating language).

In [ ]:
from nextpy.ai import engine

api_key = "sk-api_key"
model_35_turbo = engine.llms.OpenAI("gpt-3.5-turbo", chat_mode=True, api_key=api_key, caching=False)
model_davinci = engine.llms.OpenAI("text-davinci-003", chat_mode=False, api_key=api_key, caching=False)


# Basic templating

Single variable


In [ ]:
program = engine('''What is {{example}}?''')

# this program has not been executed yet, so it still has the template placeholder in it
program 

In [ ]:
# when we execute the program (by calling it) template placeholders are filled in
# note that keyword arguments to the program become variables in the template namespace
executed_program = program(example='truth')

In [ ]:
# all the variables used by the program are returned as part of the executed program
executed_program['example']

Lists and objects

In [ ]:
# define some variables we will use in the engine program
people = ['John', 'Mary', 'Bob', 'Alice']
ideas = [
    {'name': 'truth', 'description': 'the state of being the case'},
    {'name': 'love', 'description': 'a strong feeling of affection'}
]

# we can use the `each` block to iterate over a list
program = engine('''List of people:
{{#each people}}- {{this}}
{{~! This is a comment. The ~ removes adjacent whitespace either before or after a tag, depending on where you place it}}
{{/each~}}
List of ideas:
{{#each ideas}}{{this.name}}: {{this.description}}
{{/each}}''')

program(people=people, ideas=ideas)

Includes (including engine programs inside other programs)

In [ ]:
# define the program we will include
program1 = engine('''List of people:
{{#each people}}- {{this}}
{{/each~}}''')

# note that {{>prog_name}} is the same include syntax as in Handlebars
program2 = engine('''{{>program1}}
List of ideas:
{{#each ideas}}{{this.name}}: {{this.description}}
{{/each}}''')

# we can pass program just like any other variable
program2(program1=program1, people=people, ideas=ideas)

Generating text from an LLM

In [ ]:
# we can use the {{gen}} command to generate text from the language model
# note that we used a ~ at the start of the command tag to remove the whitespace before it (just like in Handlebars)
program = engine('''The best thing about the beach is {{~gen 'best' temperature=0.7 max_tokens=7}}''',llm = model_davinci)
program()

Flushing caches

In [ ]:
# you can flush a cache by calling the clear method
# (this returns the number of items that were cleared)
engine.llms.OpenAI.cache.clear()

# you can also disable caching by passing caching=False to the LLM constructor
# engine.llm = engine.llms.OpenAI("text-davinci-003", caching=False)

Selecting alternatives with the LLM

In [ ]:
# the {{#select}} command allows you to use the LLM to select from a set of options
program = engine('''Is the following sentence offensive? Please answer with a single word, either "Yes", "No", or "Maybe".
Sentence: {{example}}
Answer:{{#select "answer" logprobs='logprobs'}} Yes{{or}} No{{or}} Maybe{{/select}}''',llm = model_davinci )
executed_program = program(example='I hate tacos')

In [ ]:
# all the variables set by the program are returned as part of the executed program
executed_program['logprobs']

In [ ]:
executed_program['answer']

In [ ]:
# the example above used a block version of the select command, but you can also
# use a non-block version and just pass in a list of options
options = [' Yes', ' No', ' Maybe']
program = engine('''Is the following sentence offensive? Please answer with a single word, either "Yes", "No", or "Maybe".
Sentence: {{example}}
Answer:{{select "answer" options=options}}''',llm = model_davinci)
executed_program = program(example='I hate tacos', options=options)

In [ ]:
executed_program["answer"]

In [ ]:
executed_program['response'], executed_program['answer']

Hidden

In [ ]:
# it is often useful to execute a part of the program, but then not include that part in later context
# given to the language model. This can be done using the hidden=True argument. Several commands support
# hidden=True, but here we use the {{#block}} command (which is just a generic block command that does
# nothing other than what the arguments you pass to it do)
program = engine('''{{#block hidden=True}}Generate a response to the following email:
{{email}}.
Response:{{gen "response"}}{{/block}}
I will show you an email and a response, and you will tell me if it's offensive.
Email: {{email}}.
Response: {{response}}
Is the response above offensive in any way? Please answer with a single word, either "Yes" or "No".
Answer:{{#select "answer" logprobs='logprobs'}} Yes{{or}} No{{/select}}''',llm = model_davinci)

executed_program = program(email='I hate tacos')


Silent execution

In [ ]:
# if you want to run a program without displaying the output, you can use the silent=True argument
executed_program = program(email='I hate tacos', silent=True)
executed_program['answer']

Generating with `n>1`

In [ ]:
# the {{gen}} command the n=number argument to generate multiple completions
# only the first completion is used for future context, but the variable set
# by the command is a list of all the completions, and you can interactively
# click through each completion in the notebook visualization
program = engine('''The best thing about the beach is{{gen 'best' n=3 temperature=0.7 max_tokens=7}}''',llm = model_davinci)
executed_program = program()

In [ ]:
executed_program["best"]

Calling custom user defined functions

In [ ]:
# all the built in commands are functions from engine.library.* but you can also pass in your own functions
def aggregate(best):
    return '\n'.join(['- ' + x for x in best])

# note that we use hidden=True to prevent the {{gen}} command from being included in the output, and instead
# just use the variable it sets as an input to the aggregate function
program = engine('''The best thing about the beach is{{gen 'best' n=3 temperature=0.7 max_tokens=7 hidden=True}}
{{aggregate best}}''',llm = model_davinci)
executed_program = program(aggregate=aggregate)

Await

In [ ]:
# sometimes you want to partially execute a program, the `await` command allows you to do this
# it awaits a variable and then consumes that variables (so after the await command the variable)
prompt = engine('''Generate a response to the following email:
{{email}}.
Response:{{gen "response"}}
{{await 'instruction'}}
{{gen 'updated_response'}}''', stream=True,llm = model_davinci )

# note how the executed program is only partially executed, it stops at the await command
# because the instruction variable is not yet set
prompt = prompt(email='Hello there')


In [ ]:
prompt2 = prompt(instruction='Please translate the response above to Portuguese.',llm = model_davinci)
prompt2

In [ ]:
prompt2 = prompt(instruction='Please translate the response above to Chinese.')
prompt2

## Chat

In [ ]:
# to use role based chat tags you need a chat model, here we use gpt-3.5-turbo but you can use 'gpt-4' as well
engine.llm = engine.llms.OpenAI("gpt-3.5-turbo")

In [ ]:
# note that we enclose all of the text in one of the valid role tags for the model
# `system`, `user`, and `assistant` are just shorthand for {{#role name="system"}}...{{/role}}
# the whitepace outside the role tags is ignored by gpt-4, the whitespace inside the role tags is not
# so we use the ~ to remove the whitespace we don't want to give to the model (but want to keep in the code for clarity)
program = engine('''
{{#system~}}
You are a helpful assistant.
{{~/system}}

{{#user~}}
{{conversation_question}}
{{~/user}}

{{! this is a comment. note that we don't have to use a stop="stop_string" for the gen command below because engine infers the stop string from the role tag }}
{{#assistant~}}
{{gen 'response'}}
{{~/assistant}}''',llm = model_35_turbo)

executed_program = program(conversation_question='What is the meaning of life?')

Multistep

In [ ]:
# you can create and guide multi-turn conversations by using a series of role tags
experts = engine('''
{{#system~}}
You are a helpful assistant.
{{~/system}}

{{#user~}}
I want a response to the following question:
{{query}}
Who are 3 world-class experts (past or present) who would be great at answering this?
Please don't answer the question or comment on it yet.
{{~/user}}

{{#assistant~}}
{{gen 'experts' temperature=0 max_tokens=300}}
{{~/assistant}}

{{#user~}}
Great, now please answer the question as if these experts had collaborated in writing a joint anonymous answer.
In other words, their identity is not revealed, nor is the fact that there is a panel of experts answering the question.
If the experts would disagree, just present their different positions as alternatives in the answer itself (e.g. 'some might argue... others might argue...').
Please start your answer with ANSWER:
{{~/user}}

{{#assistant~}}
{{gen 'answer' temperature=0 max_tokens=500}}
{{~/assistant}}''',llm = model_35_turbo)
                   
experts(query='What is the meaning of life?')

With hidden

In [ ]:
# if you want the model to have some inner dialog but then not include that dialog
# in the context of later generations, you can use the {{#block}} command with hidden=True
program = engine('''
{{#system~}}
You are a helpful assistant.
{{~/system}}

{{#block hidden=True}}
{{#user~}}
Please tell me a joke
{{~/user}}

{{! note that we don't have engine controls inside the assistant role because
    the OpenAI API does not yet support that (Transformers chat models do) }}
{{#assistant~}}
{{gen 'joke'}}
{{~/assistant}}
{{~/block~}}

{{#user~}}
Is the following joke funny? Why or why not?
{{joke}}
{{~/user}}

{{#assistant~}}
{{gen 'funny'}}
{{~/assistant}}''',llm = model_35_turbo)
program()


Agents

In [ ]:
# by putting an `await` inside a `geneach` loop you can create agents that consume some
# varable, then do something and then wait for more content
program = engine('''
{{#system~}}
You are a helpful assistant
{{~/system}}

{{~#geneach 'conversation' stop=False}}
{{#user~}}
{{set 'this.user_text' (await 'user_text')}}
{{~/user}}

{{#assistant~}}
{{gen 'this.ai_text' temperature=0 max_tokens=300}}
{{~/assistant}}
{{~/geneach}}''',llm = model_35_turbo)
program = program(user_text ='hi there')

In [ ]:
# as we go through the loop we build up a conversation variable that contains the history of the conversation
# note that the last entry in the conversation variable is empty because the `await` call happens before any
# content is added to the `this` variable that represents the current item in the geneach loop
program['conversation']

In [ ]:
# here we call the agent again and the loop continues, in this case building out a conversation
program = program(user_text = 'What is the meaning of life?')

In [ ]:
program['conversation']

Using tools

The example below uses a search engine (or a mock of one) to answer user questions. The whole system is defined in a single `engine` program, but you could also break it into multiple programs and `await` external calls if you don't want the engine program to control the whole process.

In [ ]:
def is_search(completion):
    return '<search>' in completion

def search(query):
    # Fake search results
    return [{'title': 'How do I cancel a Subscription? | Facebook Help Center',
        'snippet': "To stop a monthly Subscription to a creator: Go to the creator's Facebook Page using the latest version of the Facebook app for iOS, Android or from a computer. Select Go to Supporter Hub. Select . Select Manage Subscription to go to the iTunes or Google Play Store and cancel your subscription. Cancel your Subscription at least 24 hours before ..."},
        {'title': 'News | FACEBOOK Stock Price Today | Analyst Opinions - Insider',
        'snippet': 'Stock | News | FACEBOOK Stock Price Today | Analyst Opinions | Markets Insider Markets Stocks Indices Commodities Cryptocurrencies Currencies ETFs News Facebook Inc (A) Cert Deposito Arg Repr...'},
        {'title': 'Facebook Stock Price Today (NASDAQ: META) Quote, Market Cap, Chart ...',
        'snippet': 'Facebook Stock Price Today (NASDAQ: META) Quote, Market Cap, Chart | WallStreetZen Meta Platforms Inc Stock Add to Watchlist Overview Forecast Earnings Dividend Ownership Statistics $197.81 +2.20 (+1.12%) Updated Mar 20, 2023 Meta Platforms shares are trading... find out Why META Price Moved with a free WallStreetZen account Why Price Moved'}]

search_demo = engine('''Seach results:
{{~#each results}}
<result>
{{this.title}}
{{this.snippet}}
</result>{{/each}}''')

demo_results = [
    {'title': 'OpenAI - Wikipedia', 'snippet': 'OpenAI systems run on the fifth most powerful supercomputer in the world. [5] [6] [7] The organization was founded in San Francisco in 2015 by Sam Altman, Reid Hoffman, Jessica Livingston, Elon Musk, Ilya Sutskever, Peter Thiel and others, [8] [1] [9] who collectively pledged US$ 1 billion. Musk resigned from the board in 2018 but remained a donor.'},
    {'title': 'About - OpenAI', 'snippet': 'About OpenAI is an AI research and deployment company. Our mission is to ensure that artificial general intelligence benefits all of humanity. Our vision for the future of AGI Our mission is to ensure that artificial general intelligence—AI systems that are generally smarter than humans—benefits all of humanity. Read our plan for AGI'}, 
    {'title': 'Ilya Sutskever | Stanford HAI', 'snippet': '''Ilya Sutskever is Co-founder and Chief Scientist of OpenAI, which aims to build artificial general intelligence that benefits all of humanity. He leads research at OpenAI and is one of the architects behind the GPT models. Prior to OpenAI, Ilya was co-inventor of AlexNet and Sequence to Sequence Learning.'''}
]

s = search_demo(results=demo_results)

practice_round = [
    {'role': 'user', 'content' : 'Who are the founders of OpenAI?'},
    {'role': 'assistant', 'content': '<search>Who are the founders of OpenAI</search>'},
    {'role': 'user', 'content': str(search_demo(results=demo_results))},
    {'role': 'assistant', 'content': 'The founders of OpenAI are Sam Altman, Reid Hoffman, Jessica Livingston, Elon Musk, Ilya Sutskever, Peter Thiel and others.'},
]

program = engine('''
{{#system~}}
You are a helpful assistant.
{{~/system}}

{{#user~}}
From now on, whenever your response depends on any factual information, please search the web by using the function <search>query</search> before responding. I will then paste web results in, and you can respond.
{{~/user}}

{{#assistant~}}
Ok, I will do that. Let's do a practice round
{{~/assistant}}

{{#each practice}}
{{#if (== this.role "user")}}
{{#user}}{{this.content}}{{/user}}
{{else}}
{{#assistant}}{{this.content}}{{/assistant}}
{{/if}}
{{/each}}

{{#user~}}
That was great, now let's do another one.
{{~/user}}

{{#assistant~}}
Sounds good
{{~/assistant}}

{{#user~}}
{{user_query}}
{{~/user}}

{{#assistant~}}
{{gen "query" stop="</search>"}}{{#if (is_search query)}}</search>{{/if}}
{{~/assistant}}

{{#user~}}
Search results: {{#each (search query)}}
<result>
{{this.title}}
{{this.snippet}}
</result>{{/each}}
{{~/user}}

{{#assistant~}}
{{gen "answer"}}
{{~/assistant}}
''',llm = model_35_turbo)

query = "What is Facebook's stock price right now?"

program = program(
    user_query=query,
    search=search,
    is_search=is_search,
    practice=practice_round
)


<hr style="height: 1px; opacity: 0.5; border: none; background: #cccccc;">
<div style="text-align: center; opacity: 0.5">Have an idea for more helpful examples? Pull requests that add to this documentation notebook are encouraged!</div>